In [1]:
# Import 

import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *

#SET BOARD SIZE
size_board = 4

In [2]:
##INITIALISE A GAME 
env=Chess_Env(size_board)
SS,XS,allowed_aS=env.Initialise_game()
N_actions = len(allowed_aS)

In [3]:
##IMPLEMENT EPSILON GREEDY POLICY
def EpsilonGreedy_Policy(Q, epsilon):

    rand_value=np.random.uniform(0,1)

    rand_a=rand_value<epsilon
    
    aS,_=np.where(allowed_a==1)
    if rand_a==True:
        
        a=np.random.choice(aS)

    else:

        maxQ=np.argmax(Q)
        a = aS[maxQ]
            
    return a

In [4]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 

class Model:
    def __init__(self):
        self.layers = []
        self.loss = []   
    def add(self, layer):
        self.layers.append(layer)
    def predict(self, X):
        # Forward pass
        for i, _ in enumerate(self.layers):
            forward = self.layers[i].forward(X)
            X = forward          
        return forward

    def update_w(self, X, Y, learning_rate):
        # Forward pass
        for i, _ in enumerate(self.layers):
            forward = self.layers[i].forward(input_val=X)
            X = forward         
        # Compute loss and first gradient
        mse = MeanSquaredError(forward, Y)
        error = mse.forward()
        gradient = mse.backward()  
        self.loss.append(error)
         
        # Backpropagation
        for i, _ in reversed(list(enumerate(self.layers))):
            if self.layers[i].type != 'Linear':
                gradient = self.layers[i].backward(gradient)
            else:
                gradient, dW, dB = self.layers[i].backward(gradient)
                self.layers[i].optimize(dW, dB, learning_rate)             
        return error


class Layer: #Layer abstract class
    def __init__(self):
        pass   
    def __len__(self):
        pass     
    def __str__(self):
        pass   
    def forward(self):
        pass   
    def backward(self):
        pass  
    def optimize(self):
        pass
        
class Linear(Layer):
    def __init__(self, input_dim, output_dim):
        np.random.seed(90)
        self.weights = np.random.rand(output_dim, input_dim)
        np.random.seed(90)
        self.biases = np.random.rand(output_dim, 1)
        self.type = 'Linear'
    def __str__(self):
        return f"{self.type} Layer"     
    def forward(self, input_val):
        self._prev_acti = input_val.reshape(self.weights.shape[1],1)
        return np.matmul(self.weights, input_val).reshape(self.weights.shape[0],1) + self.biases   
    def backward(self, dA):
        dW = np.dot(dA, self._prev_acti.T)
        dB = dA.mean(axis=1, keepdims=True)        
        delta = np.dot(self.weights.T, dA)        
        return delta, dW, dB    
    def optimize(self, dW, dB, rate):
        self.weights = self.weights - rate * dW
        self.biases = self.biases - rate * dB  
        
class ReLU(Layer):    
    def __init__(self, output_dim):
        self.units = output_dim
        self.type = 'ReLU'
    def __str__(self):
        return f"{self.type} Layer"               
    def forward(self, input_val):
        self._prev_acti = np.maximum(0, input_val)
        return self._prev_acti    
    def backward(self, dJ):
        return dJ * np.heaviside(self._prev_acti, 0)

class Sigmoid(Layer):
    def __init__(self, output_dim):
        self.units = output_dim
        self.type = 'Sigmoid'
    def __str__(self):
        return f"{self.type} Layer"                
    def forward(self, input_val):
        self._prev_acti = 1 / (1 + np.exp(-input_val))
        return self._prev_acti    
    def backward(self, dJ):
        sig = self._prev_acti
        return dJ * sig * (1 - sig)
    
class MeanSquaredError(Layer):
    def __init__(self, predicted, real):
        self.predicted = predicted
        self.real = real
        self.type = 'Mean Squared Error'
    def forward(self):
        return np.power(self.predicted - self.real, 2).mean()
    def backward(self):
        return 2 * (self.predicted - self.real)/self.predicted.size

In [5]:
# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
gamma_2 = 0.9         #Here we try different Gamma Values
gamma_3 = 0.7 
gamma_4 = 0.6
eta = 0.0035        # THE LEARNING RATE


N_episodes=40000 # THE NUMBER OF GAMES TO BE PLAYED 

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_ql = np.zeros([N_episodes, 1])
N_moves_save_ql = np.zeros([N_episodes, 1])
R_save_ql_gamma_2 = np.zeros([N_episodes, 1])
N_moves_save_ql_gamma_2 = np.zeros([N_episodes, 1])
R_save_ql_gamma_3= np.zeros([N_episodes, 1])
N_moves_save_ql_gamma_3 = np.zeros([N_episodes, 1])
R_save_ql_gamma_4 = np.zeros([N_episodes, 1])
N_moves_save_ql_gamma_4 = np.zeros([N_episodes, 1])

In [6]:
# EXPONENTIAL MOVING AVERAGE: B = [0,1]
def EMA(Y,B):
    y = 0
    EMAY = []
    for n in Y:
        y = B * y + (1 - B) * n
        EMAY.append(y.copy())
    return EMAY

In [7]:
## INITALISE YOUR NEURAL NETWORK...
## NUMBER OF HIDDEN NODES
N_h=200
# Create model
model = Model() 
# Add layers
model.add(Linear(np.shape(XS)[0], N_h))
model.add(ReLU(N_h))
model.add(Linear(N_h,32))
model.add(ReLU(32))

In [8]:
#TRAINING STRUCTURE
for n in range(N_episodes):
    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        Qvaluesall = model.predict(X.reshape(58,1))
        Qvaluesactions = Qvaluesall[np.where(allowed_a==1)[0]]
        a_agent=EpsilonGreedy_Policy(Qvaluesactions, epsilon_f)
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        if Done:
            
            R_save_ql[n]=np.copy(R)
            N_moves_save_ql[n]=np.copy(i)
            Qvaluesall[a_agent] = R
            model.update_w(X,Qvaluesall,eta)
            break
            
        # IF THE EPISODE IS NOT OVER...    
        else:
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE... 
            Qvaluesall_next = model.predict(X_next.reshape(58,1)) 
            Qvaluesactions_next = Qvaluesall_next[np.where(allowed_a==1)[0]]
            a_agent_next = EpsilonGreedy_Policy(Qvaluesactions_next,0)
            Qvaluesall[a_agent] = R + gamma*Qvaluesall_next[a_agent_next]
            model.update_w(X,Qvaluesall,eta)

            S=np.copy(S_next)
            X=np.copy(X_next)
            allowed_a=np.copy(allowed_a_next)

        # UPDATE THE COUNTER
        i=i+1
    #call back
    if n%500 ==0:
        print("Have Finished "+ str(n)+" Epoachs")
        

Have Finished 0 Epoachs
Have Finished 500 Epoachs
Have Finished 1000 Epoachs
Have Finished 1500 Epoachs
Have Finished 2000 Epoachs


KeyboardInterrupt: 

In [ ]:
## INITALISE YOUR NEURAL NETWORK...
## NUMBER OF HIDDEN NODES
N_h=200
# Create model
model_2 = Model() 
# Add layers
model_2.add(Linear(np.shape(XS)[0], N_h))
model_2.add(ReLU(N_h))
model_2.add(Linear(N_h,32))
model_2.add(ReLU(32))

In [ ]:
#TRAINING STRUCTURE
for n in range(N_episodes):
    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        Qvaluesall = model_2.predict(X.reshape(58,1))
        Qvaluesactions = Qvaluesall[np.where(allowed_a==1)[0]]
        a_agent=EpsilonGreedy_Policy(Qvaluesactions, epsilon_f)
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        if Done:
            
            R_save_ql_gamma_2[n]=np.copy(R)
            N_moves_save_ql_gamma_2[n]=np.copy(i)
            Qvaluesall[a_agent] = R
            model_2.update_w(X,Qvaluesall,eta)
            break
            
        # IF THE EPISODE IS NOT OVER...    
        else:
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE... 
            Qvaluesall_next = model_2.predict(X_next.reshape(58,1)) 
            Qvaluesactions_next = Qvaluesall_next[np.where(allowed_a==1)[0]]
            a_agent_next = EpsilonGreedy_Policy(Qvaluesactions_next,0)
            Qvaluesall[a_agent] = R + gamma_2*Qvaluesall_next[a_agent_next]
            model_2.update_w(X,Qvaluesall,eta)

            S=np.copy(S_next)
            X=np.copy(X_next)
            allowed_a=np.copy(allowed_a_next)

        # UPDATE THE COUNTER
        i=i+1
    #call back
    if n%500 ==0:
        print("Have Finished "+ str(n)+" Epoachs")

In [ ]:
## INITALISE YOUR NEURAL NETWORK...
## NUMBER OF HIDDEN NODES
N_h=200
# Create model
model_3 = Model() 
# Add layers
model_3.add(Linear(np.shape(XS)[0], N_h))
model_3.add(ReLU(N_h))
model_3.add(Linear(N_h,32))
model_3.add(ReLU(32))

In [ ]:
#TRAINING STRUCTURE
for n in range(N_episodes):
    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        Qvaluesall = model_3.predict(X.reshape(58,1))
        Qvaluesactions = Qvaluesall[np.where(allowed_a==1)[0]]
        a_agent=EpsilonGreedy_Policy(Qvaluesactions, epsilon_f)
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        if Done:
            
            R_save_ql_gamma_3[n]=np.copy(R)
            N_moves_save_ql_gamma_3[n]=np.copy(i)
            Qvaluesall[a_agent] = R
            model_3.update_w(X,Qvaluesall,eta)
            break
            
        # IF THE EPISODE IS NOT OVER...    
        else:
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE... 
            Qvaluesall_next = model_3.predict(X_next.reshape(58,1)) 
            Qvaluesactions_next = Qvaluesall_next[np.where(allowed_a==1)[0]]
            a_agent_next = EpsilonGreedy_Policy(Qvaluesactions_next, 0)
            Qvaluesall[a_agent] = R + gamma_3*Qvaluesall_next[a_agent_next]
            model_3.update_w(X,Qvaluesall,eta)

            S=np.copy(S_next)
            X=np.copy(X_next)
            allowed_a=np.copy(allowed_a_next)

        # UPDATE THE COUNTER
        i=i+1
    #call back
    if n%500 ==0:
        print("Have Finished "+ str(n)+" Epoachs")

In [ ]:
## INITALISE YOUR NEURAL NETWORK...
## NUMBER OF HIDDEN NODES
N_h=200
# Create model
model_4 = Model() 
# Add layers
model_4.add(Linear(np.shape(XS)[0], N_h))
model_4.add(ReLU(N_h))
model_4.add(Linear(N_h,32))
model_4.add(ReLU(32))

In [ ]:
#TRAINING STRUCTURE
for n in range(N_episodes):
    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        Qvaluesall = model_4.predict(X.reshape(58,1))
        Qvaluesactions = Qvaluesall[np.where(allowed_a==1)[0]]
        a_agent=EpsilonGreedy_Policy(Qvaluesactions, epsilon_f)
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        if Done:
            
            R_save_ql_gamma_4[n]=np.copy(R)
            N_moves_save_ql_gamma_4[n]=np.copy(i)
            Qvaluesall[a_agent] = R
            model_4.update_w(X,Qvaluesall,eta)
            break
            
        # IF THE EPISODE IS NOT OVER...    
        else:
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE... 
            Qvaluesall_next = model_4.predict(X_next.reshape(58,1)) 
            Qvaluesactions_next = Qvaluesall_next[np.where(allowed_a==1)[0]]
            a_agent_next = EpsilonGreedy_Policy(Qvaluesactions_next, 0)
            Qvaluesall[a_agent] = R + gamma_4*Qvaluesall_next[a_agent_next]
            model_4.update_w(X,Qvaluesall,eta)

            S=np.copy(S_next)
            X=np.copy(X_next)
            allowed_a=np.copy(allowed_a_next)

        # UPDATE THE COUNTER
        i=i+1
    #call back
    if n%500 ==0:
        print("Have Finished "+ str(n)+" Epoachs")

In [ ]:
## PLOT THE TOTAL REWARD PER EPISODE AS TRAINING PROGRESSES
plt.figure()
plt.plot(EMA(R_save_ql,0.999), color='green', label="gamma = 0.85")
plt.plot(EMA(R_save_ql_gamma_2,0.999), color='red', label="gamma = 0.9")
plt.plot(EMA(R_save_ql_gamma_3,0.999), color='black', label="gamma = 0.7")
plt.plot(EMA(R_save_ql_gamma_4,0.999), color='blue', label="gamma = 0.6")
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.title("Q-Learning")
plt.legend()
plt.show()

In [ ]:
## PLOT THE NUMBER OF MOVES AS TRAINING PROGRESSES
plt.figure()
plt.plot(EMA(N_moves_save_ql,0.9995), color='green', label="gamma = 0.85")
#plt.plot(EMA(N_moves_save_ql_gamma_2,0.9995), color='red', label="gamma = 0.9")
plt.plot(EMA(N_moves_save_ql_gamma_3,0.9995), color='black', label="gamma = 0.7")
plt.plot(EMA(N_moves_save_ql_gamma_4,0.9995), color='blue', label="gamma = 0.6")
plt.xlabel('Episode Number')
plt.ylabel('Number of Moves')
plt.title("Q-Learning")
plt.legend()
plt.show()